Домашнее задание к занятию "Алгоритмы классификации: линейные методы, логистическая регрессия и SVM"

Имеются данные adult.csv

Целевой переменной является уровень дохода income (крайний правый столбец).

Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

Hеобходимо построить модель логистической регрессии, которая предсказывает уровень дохода человека. При возможности попробуйте улучшить точность предсказаний (метод score) с помощью перебора признаков.

In [114]:
import pandas as pd
data = pd.read_csv("adult.csv")
print(data.shape)
data.head()

(48842, 15)


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [115]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


48842 строк х 15 столбцов

Пропущенных значений нет

Необходимо адаптировать признаки - только 6 из 15 являются числовыми

In [116]:
from sklearn.preprocessing import LabelEncoder

In [117]:
#Преобразую целевую переменную income, которая имеет всего 2 значения - в 0/1
le_income = LabelEncoder()
le_income.fit(data['income'])
y = pd.Series(data = le_income.transform(data['income']))
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [118]:
del data["income"]

In [119]:
# 1) Workclass показывает класс занятости человека - может напрямую указывать на степень его доходов
# Видим также, что есть пропущенные знчения, которые обозначены "?"
data["workclass"].unique()

array(['Private', 'Local-gov', '?', 'Self-emp-not-inc', 'Federal-gov',
       'State-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [120]:
# Переведем категориальную переменную Workclass в 0/1 при помощи get_dummies
data_1 = pd.get_dummies(data, columns = ['workclass'])
data_1.head()

,age,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,...,native-country,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,25,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,...,United-States,0,0,0,0,1,0,0,0,0
1,38,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,...,United-States,0,0,0,0,1,0,0,0,0
2,28,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,...,United-States,0,0,1,0,0,0,0,0,0
3,44,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,...,United-States,0,0,0,0,1,0,0,0,0
4,18,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,...,United-States,1,0,0,0,0,0,0,0,0


In [121]:
# По той же логике уровень образования влияет на заработок, перемененнная категориальная
data["education"].unique()

array(['11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th',
       'Prof-school', '7th-8th', 'Bachelors', 'Masters', 'Doctorate',
       '5th-6th', 'Assoc-voc', '9th', '12th', '1st-4th', 'Preschool'],
      dtype=object)

In [122]:
# Как видим, переменная educational-num является аналогом поля education, причем еще и ранжирует все
# значения по значимости образования. Попробуем использовать эту переменную, а education - drop.
data[["education", "educational-num"]].groupby(["education"]).mean().sort_values("educational-num")

,educational-num
education,
Preschool,1
1st-4th,2
5th-6th,3
7th-8th,4
9th,5
10th,6
11th,7
12th,8
HS-grad,9


In [123]:
del data_1["education"]
data_1.head()

,age,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,...,native-country,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,25,226802,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,...,United-States,0,0,0,0,1,0,0,0,0
1,38,89814,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,...,United-States,0,0,0,0,1,0,0,0,0
2,28,336951,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,...,United-States,0,0,1,0,0,0,0,0,0
3,44,160323,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,...,United-States,0,0,0,0,1,0,0,0,0
4,18,103497,10,Never-married,?,Own-child,White,Female,0,0,...,United-States,1,0,0,0,0,0,0,0,0


Имеются еще несколько категориальных переменных:

1) marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
- признак косвенно может влиять на зароботок, преобразую с помощью get_dummies, попробую обучить с ним и без него

2) occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- признак (по идее) напрямую влияет на заработок, преобразую с помощью get_dummies

3) relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- может дублировать п.1, причем более точно, ведь наличие детей может говорить о необходимоти большего заработка. В данном случае Wife = Husband (пол учтен отдельно)

4) race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
- может влиять на заработок, преобразую с помощью get_dummies

5) sex: Female, Male.
- может влиять на заработок, преобразую с помощью LabelEncoder

6) native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- скорее всего влияние данного признака будет учтено в п. 4. Попробую оценить вклад признака с помощью перевода его в get_dummies

->Попробую различные варианты построения логистической регресии в зависимости от выбранных признаков

In [124]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

**Попытка 1. Только изначально числовые переменные**

In [125]:
data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

In [126]:
Columns = ['age', 'fnlwgt', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']
X = data[Columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [127]:
X.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape

((48842, 6), (39073, 6), (9769, 6), (39073,), (9769,))

In [128]:
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,25,226802,7,0,0,40
1,38,89814,9,0,0,50
2,28,336951,12,0,0,40
3,44,160323,10,7688,0,40
4,18,103497,10,0,0,30


In [129]:
model = LogisticRegression(C = 1) #тут надо играть с C

In [130]:
# обучаем модель
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Users\Anastasia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [131]:
y_pred[:8]

array([0, 0, 0, 0, 0, 0, 0, 1])

In [132]:
y_test[:8]

20707    0
26175    1
24476    1
5434     0
31609    0
16199    0
43738    0
18924    1
dtype: int32

In [133]:
model.score(X, y)

0.79767413291839

**Попытка 2. Добавлю половой признак**

In [134]:
Columns = ['age', 'fnlwgt', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week', 'gender']
X = data[Columns]
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,gender
0,25,226802,7,0,0,40,Male
1,38,89814,9,0,0,50,Male
2,28,336951,12,0,0,40,Male
3,44,160323,10,7688,0,40,Male
4,18,103497,10,0,0,30,Female


In [135]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit( X['gender'] )
X['gender'] = le.transform( X['gender'] )
X.head()

C:\Users\Anastasia\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,gender
0,25,226802,7,0,0,40,1
1,38,89814,9,0,0,50,1
2,28,336951,12,0,0,40,1
3,44,160323,10,7688,0,40,1
4,18,103497,10,0,0,30,0


In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(C = 1) #тут надо играть с C
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X, y)

C:\Users\Anastasia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7974693910978257

**score не увеличился**

**Попытка 3. Добавлю признак workclass**

In [137]:
del X["gender"]
X["workclass"] = data["workclass"]
X.head()

C:\Users\Anastasia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass
0,25,226802,7,0,0,40,Private
1,38,89814,9,0,0,50,Private
2,28,336951,12,0,0,40,Local-gov
3,44,160323,10,7688,0,40,Private
4,18,103497,10,0,0,30,?


In [138]:
X = pd.get_dummies(X, columns = ['workclass'])
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,25,226802,7,0,0,40,0,0,0,0,1,0,0,0,0
1,38,89814,9,0,0,50,0,0,0,0,1,0,0,0,0
2,28,336951,12,0,0,40,0,0,1,0,0,0,0,0,0
3,44,160323,10,7688,0,40,0,0,0,0,1,0,0,0,0
4,18,103497,10,0,0,30,1,0,0,0,0,0,0,0,0


In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(C = 1) #тут надо играть с C
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X, y)

C:\Users\Anastasia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7984316776544778

**Ура! score увеличился**

**Попытка 4. Добавлю признак marital-status**

In [140]:
X["marital-status"] = data["marital-status"]
X = pd.get_dummies(X, columns = ['marital-status'])
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed
0,25,226802,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,28,336951,12,0,0,40,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,44,160323,10,7688,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,18,103497,10,0,0,30,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [141]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(C = 1) #тут надо играть с C
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X, y)

C:\Users\Anastasia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.79767413291839

**score снизился**

**Попытка 5. Добавлю признак occupation**

Значения: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

признак (по идее) напрямую влияет на заработок, преобразую с помощью get_dummies

In [142]:
X.columns

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_?', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay',
       'marital-status_Divorced', 'marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed'],
      dtype='object')

In [146]:
X.columns[:15]

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_?', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay'],
      dtype='object')

In [147]:
X = X[X.columns[:15]]
X["occupation"] = data["occupation"]
X = pd.get_dummies(X, columns = ['occupation'])
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,25,226802,7,0,0,40,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,38,89814,9,0,0,50,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,28,336951,12,0,0,40,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,44,160323,10,7688,0,40,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,18,103497,10,0,0,30,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(C = 1) #тут надо играть с C
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X, y)

C:\Users\Anastasia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7976536587363335

**score стал ниже**

**Попытка 6. Добавлю признак relationship**

In [149]:
X = X[X.columns[:15]]
X["relationship"] = data["relationship"]
X = pd.get_dummies(X, columns = ['relationship'])
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,25,226802,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,28,336951,12,0,0,40,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,44,160323,10,7688,0,40,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,18,103497,10,0,0,30,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(C = 1) #тут надо играть с C
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X, y)

C:\Users\Anastasia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7980426681954056

**score понизился**

**Попытка 7. Добавлю признак race**

In [151]:
X.columns

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_?', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay', 'relationship_Husband',
       'relationship_Not-in-family', 'relationship_Other-relative',
       'relationship_Own-child', 'relationship_Unmarried',
       'relationship_Wife'],
      dtype='object')

In [152]:
X = X[X.columns[:15]]
X["race"] = data["race"]
X = pd.get_dummies(X, columns = ['race'])
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,226802,7,0,0,40,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,28,336951,12,0,0,40,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,44,160323,10,7688,0,40,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,18,103497,10,0,0,30,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [153]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(C = 1) #тут надо играть с C
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X, y)

C:\Users\Anastasia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7981450391056877

**score понизился**

**Последняя попытка 8. Добавлю native-country**

In [155]:
X = X[X.columns[:15]]
X["native-country"] = data["native-country"]
X = pd.get_dummies(X, columns = ['native-country'])
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,7688,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,0,0,30,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(C = 1) #тут надо играть с C
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X, y)

C:\Users\Anastasia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7981450391056877

**score опять уменьшился. Оставляем за результат попытку 3.**

**Хотя надо отметить, что результат попытки 3 немногим превышает результат попытки 1 и может свидетельствовать лишь о более удачном случайном разбиении выборки на train-test**